In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json

import shapely
from shapely.geometry import mapping, shape, Point, LineString, MultiLineString, Polygon
import pandas as pd
import numpy as np
import geojsonio
import geopandas as gpd
import utm

gtfs_tk_dir = '../../gtfs-tk/'
sys.path.append(gtfs_tk_dir)
import gtfs_tk.feed as gf
import gtfs_tk.utils as gu

%load_ext autoreload
%autoreload 2
%matplotlib inline  


In [2]:
feed = gf.Feed(gtfs_tk_dir + 'data/portland_gtfs.zip', original_units='ft')
#feed = gf.Feed(gtfs_tk_dir + 'data/cairns_gtfs.zip')

# Set study date
date = feed.get_dates()[0]

print(feed.utm_crs)

{'units': 'm', 'proj': 'utm', 'ellps': 'WGS84', 'zone': '10T'}


In [3]:
feed.routes.head()

,route_id,route_short_name,route_long_name,route_type,route_url,route_sort_order
0,1,1,Vermont,3,http://trimet.org/schedules/r001.htm,400
1,4,4,Division/Fessenden,3,http://trimet.org/schedules/r004.htm,600
2,6,6,Martin Luther King Jr Blvd,3,http://trimet.org/schedules/r006.htm,800
3,8,8,Jackson Park/NE 15th,3,http://trimet.org/schedules/r008.htm,1000
4,9,9,Powell Blvd,3,http://trimet.org/schedules/r009.htm,1100


In [21]:
def get_route_geojson(feed, route_id):
    # Get trips
    trips = feed.trips[feed.trips['route_id'] == route_id]

    # Get shapes
    shapes = pd.merge(feed.shapes, trips[['trip_id', 'shape_id']]) 
    geom = MultiLineString(shapes['geometry'].simplify().values.tolist())
                         
    # Make route feature
    route = feed.routes[feed.routes['route_id'] == route_id]
    route_features = [{
      'type': 'Feature', 
      'geometry': mapping(geom),
      'properties': json.loads(route.to_json()), 
      }]
    
    # Get stops
    stop_ids = pd.merge(feed.stop_times, trips)['stop_id'].unique()
    stops = feed.stops[feed.stops['stop_id'].isin(stop_ids)].copy()
    stop_cols = list(stops.columns)
    stop_cols.remove('geometry')
    
    # Make stop features
    stop_features = [{
      'type': 'Feature', 
      'geometry': mapping(row['geometry']),
      'properties': json.loads(row[stop_cols].to_json()), 
      } for index, row in stops.iterrows()]
    
    # Make feature collection
    g = {'type': 'FeatureCollection', 'features': route_features + stop_features}
    return g

route_id = '4'
# g = get_route_geojson(feed, route_id)
# print(len(g['features']))
# geojsonio.embed(json.dumps(g))

print(feed.shapes.crs)
trips = feed.trips[feed.trips['route_id'] == route_id]
shapes = pd.merge(feed.shapes, trips[['trip_id', 'shape_id']]) 
print(shapes.geometry.crs)

{'no_defs': True, 'proj': 'longlat', 'datum': 'WGS84', 'ellps': 'WGS84'}


AttributeError: 'Series' object has no attribute 'crs'

In [213]:
# Test screen line
s = Polygon(shape(
        {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.68033504486084,
              45.49007323249593
            ],
            [
              -122.68033504486084,
              45.490855438757684
            ],
            [
              -122.67921924591063,
              45.490855438757684
            ],
            [
              -122.67921924591063,
              45.49007323249593
            ],
            [
              -122.68033504486084,
              45.49007323249593
            ]
          ]
        ]
      }    
    ))
s
gu.time_it(feed.get_trips_intersections)(s, date)
#['route_id'].unique()

Timing get_trips_intersections
2015-04-06 13:21:19.166325 Began process
2015-04-06 13:21:20.566997 Finished in 0.02 min


,route_id,service_id,trip_id,direction_id,block_id,shape_id,trip_type,intersection
0,12,U.387,4512203,0,9367,198005,NaN,LINESTRING (-122.6792192459106 45.490645379388...
1,12,U.387,4512246,0,1204,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
2,12,U.387,4512245,0,1203,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
3,12,U.387,4512244,0,1268,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
4,12,U.387,4512243,0,1267,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
5,12,U.387,4512242,0,1204,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
6,12,U.387,4512241,0,1203,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
7,12,U.387,4512240,0,1268,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
8,12,U.387,4512238,0,1267,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...
9,12,U.387,4512236,0,1201,198007,NaN,LINESTRING (-122.6792192459106 45.490645379388...


In [4]:
trips_stats = feed.get_trips_stats()

In [11]:
trips = feed.get_trips(date)

# Compute peak stats
f = pd.merge(trips, trips_stats)
f[['start_time', 'end_time']] =\
  f[['start_time', 'end_time']].applymap(gu.timestr_to_seconds)

d = {}
d['time'] =np.unique(f[['start_time', 'end_time']].values)
d['count'] = [gf.count_active_trips(f, t) for t in d['time']]
g = pd.DataFrame(d)
g
m = g['count'].max()
g[g['count'] == m]
#start, end = utils.get_peak_indices(times, counts)
# d['peak_num_trips'] = counts[start]
# d['peak_start_time'] =\
#   utils.timestr_to_seconds(times[start], inverse=True)
# d['peak_end_time'] =\
#   utils.timestr_to_seconds(times[end], inverse=True)


,count,time
79,39,29760
99,39,31560
445,39,63960


In [13]:
route = '110-423'
f = routes_stats
f[f['route_id'] == route].T

,0
route_id,110-423
num_trips,34
is_loop,0
is_bidirectional,1
start_time,06:16:00
end_time,25:04:00
max_headway,60
mean_headway,60
peak_num_trips,2
peak_start_time,08:16:00


In [75]:
f = routes_ts
f = f['num_trips'][route]
f.ix[8*60:(9*60 + 5)]


2014-06-07 08:00:00    1
2014-06-07 08:01:00    1
2014-06-07 08:02:00    1
2014-06-07 08:03:00    1
2014-06-07 08:04:00    1
2014-06-07 08:05:00    1
2014-06-07 08:06:00    1
2014-06-07 08:07:00    1
2014-06-07 08:08:00    2
2014-06-07 08:09:00    2
2014-06-07 08:10:00    1
2014-06-07 08:11:00    1
2014-06-07 08:12:00    1
2014-06-07 08:13:00    1
2014-06-07 08:14:00    1
2014-06-07 08:15:00    1
2014-06-07 08:16:00    2
2014-06-07 08:17:00    2
2014-06-07 08:18:00    2
2014-06-07 08:19:00    2
2014-06-07 08:20:00    2
2014-06-07 08:21:00    2
2014-06-07 08:22:00    2
2014-06-07 08:23:00    2
2014-06-07 08:24:00    2
2014-06-07 08:25:00    2
2014-06-07 08:26:00    2
2014-06-07 08:27:00    2
2014-06-07 08:28:00    2
2014-06-07 08:29:00    2
                      ..
2014-06-07 08:35:00    2
2014-06-07 08:36:00    2
2014-06-07 08:37:00    2
2014-06-07 08:38:00    2
2014-06-07 08:39:00    2
2014-06-07 08:40:00    2
2014-06-07 08:41:00    2
2014-06-07 08:42:00    2
2014-06-07 08:43:00    2


In [29]:
date = feed.get_dates()[0]
f = gu.time_it(feed.get_routes_time_series)(trips_stats, date, freq='5Min')
print(f.index)
f.to_csv('temp.csv')
g = pd.read_csv('temp.csv', header=[0, 1], parse_dates=True, index_col=0, infer_datetime_format=True)
x = pd.tseries.frequencies.infer_freq(g.index)
g.index.freq = pd.tseries.frequencies.to_offset(x)
print(g.index)
gf.downsample(g, freq='15Min')

Timing get_routes_time_series
2015-03-27 16:36:32.265245 Began process
2015-03-27 16:36:32.457691 Finished in 0.00 min
<class 'pandas.tseries.index.DatetimeIndex'>
[2014-05-26 00:00:00, ..., 2014-05-26 23:55:00]
Length: 288, Freq: 5T, Timezone: None
<class 'pandas.tseries.index.DatetimeIndex'>
[2014-05-26 00:00:00, ..., 2014-05-26 23:55:00]
Length: 288, Freq: 5T, Timezone: None


indicator           num_trip_starts                                           \
route_id                    110-423 111-423 112-423 113-423 120-423 120N-423   
2014-05-26 00:00:00               0       0       0       0       0        0   
2014-05-26 00:15:00               0       0       0       0       0        0   
2014-05-26 00:30:00               0       0       0       0       0        0   
2014-05-26 00:45:00               0       0       0       0       0        0   
2014-05-26 01:00:00               0       0       0       0       0        0   
2014-05-26 01:15:00               0       0       0       0       0        0   
2014-05-26 01:30:00               0       0       0       0       0        0   
2014-05-26 01:45:00               0       0       0       0       0        0   
2014-05-26 02:00:00               0       0       0       0       0        0   
2014-05-26 02:15:00               0       0       0       0       0        0   
2014-05-26 02:30:00               0       0       0       0       0        0   
2014-05-26 02:45:00               0       0       0       0       0        0   
2014-05-26 03:00:00               0       0       0       0       0        0   
2014-05-26 03:15:00               0       0       0       0       0        0   
2014-05-26 03:30:00               0       0       0       0       0        0   
2014-05-26 03:45:00               0       0       0       0       0        0   
2014-05-26 04:00:00               0       0       0       0       0        0   
2014-05-26 04:15:00               0       0       0       0       0        0   
2014-05-26 04:30:00               0       0       0       0       0        0   
2014-05-26 04:45:00               0       0       0       0       0        0   
2014-05-26 05:00:00               0       0       0       0       0        0   
2014-05-26 05:15:00               0       0       0       0       0        0   
2014-05-26 05:30:00               0       0       0       0       1        0   
2014-05-26 05:45:00               1       0       0       0       0        0   
2014-05-26 06:00:00               0       1       0       1       0        0   
2014-05-26 06:15:00               1       0       0       0       0        0   
2014-05-26 06:30:00               0       1       0       1       1        0   
2014-05-26 06:45:00               1       1       0       0       0        0   
2014-05-26 07:00:00               1       0       0       0       1        0   
2014-05-26 07:15:00               1       2       0       1       0        0   
...                             ...     ...     ...     ...     ...      ...   
2014-05-26 16:30:00               1       1       0       0       1        0   
2014-05-26 16:45:00               1       1       1       0       0        0   
2014-05-26 17:00:00               1       1       0       1       1        0   
2014-05-26 17:15:00               1       1       0       0       0        0   
2014-05-26 17:30:00               1       1       0       0       1        0   
2014-05-26 17:45:00               1       1       1       0       0        0   
2014-05-26 18:00:00               2       0       0       1       1        0   
2014-05-26 18:15:00               0       1       0       0       0        0   
2014-05-26 18:30:00               1       1       0       0       1        0   
2014-05-26 18:45:00               0       1       1       0       0        0   
2014-05-26 19:00:00               2       0       0       0       1        0   
2014-05-26 19:15:00               0       0       0       0       0        0   
2014-05-26 19:30:00               0       2       0       0       1        0   
2014-05-26 19:45:00               0       0       1       0       0        0   
2014-05-26 20:00:00               2       0       0       0       1        0   
2014-05-26 20:15:00               0       0       0       0       0        0   
2014-05-26 20:30:00               0       2       0       0       1        0   
